In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
!unzip /content/drive/MyDrive/ViFLAVOR_Belief-merging/data_test.zip -d /content/
!pip install unidecode ultralytics==8.2.0

In [ ]:
import ultralytics
print(ultralytics.__version__)

8.2.0


# Belief Merging of Max

In [ ]:
from ultralytics import YOLO, RTDETR
import cv2
import math
import pandas as pd
from unidecode import unidecode
import numpy as np
import os

# Đọc file cấu hình
classes_path = '/content/drive/MyDrive/ViFLAVOR_Belief-merging/classes.txt'
excel_file_path = '/content/drive/MyDrive/ViFLAVOR_Belief-merging/rules.xlsx'

# Tải các mô hình
model_yolov8 = YOLO('/content/drive/MyDrive/ViFLAVOR_Belief-merging/yolov8_best_862.pt')
model_yolov9 = YOLO('/content/drive/MyDrive/ViFLAVOR_Belief-merging/yolov9_best_85.pt')
model_rtdetr = RTDETR('/content/drive/MyDrive/ViFLAVOR_Belief-merging/rt-detr_best_865.pt')

df = pd.read_excel(excel_file_path)
classes_ingre = []

with open(classes_path, 'r', encoding='utf-8') as file:
    lines = [line.strip() for line in file]
    classes_ingre = lines

def convertIdx2Class(set_index, classes):
    result_set = {classes[item] for item in set_index}
    return result_set

def preprocess_dataframe():
    global df
    normalized_df = df.drop(df.columns[[0, -1]], axis=1)

    for index, row in normalized_df.iterrows():
        for col in normalized_df.columns:
            value = row[col]
            if pd.notna(value):
                name_without_accents = unidecode(value)
                values = name_without_accents.split(', ')
                for i in range(len(values)):
                    values[i] = values[i].replace(' ', '-')
                row[col] = ', '.join(values)

    return normalized_df

normalized_df = preprocess_dataframe()

def detect_ingredients(model, img):
    results = model(img, conf=0.3, save=False)
    names = results[0].names
    detected_cls = results[0].boxes.cls.tolist()
    boxes = results[0].boxes.xyxy.tolist()
    return names, detected_cls, boxes

def find_foodname_ver_narrow(set_detected_ingre):
    global normalized_df
    global df
    distances = []

    for index, row in normalized_df.iterrows():
        values = {}
        values["prior"] = set(row["Priorities"].split(', '))
        values["main"] = set(row["Ingredients"].split(', '))
        values["extra"] = set(row["Secondary Ingredients"].split(', ')) if pd.notna(row["Secondary Ingredients"]) else set()

        # Không khớp với ƯU TIÊN
        no_match_prior = values["prior"].difference(set_detected_ingre)

        # Không khớp với CHÍNH
        no_match_main = values["main"].difference(set_detected_ingre)

        # Khớp với PHỤ
        match_extra = values["extra"].intersection(set_detected_ingre)

        # Phần tử thuộc detect nhưng không thuộc bộ luật CHÍNH & PHỤ
        combine = values["prior"].union(values["main"])
        combine = combine.union(values["extra"])
        redundancy = set_detected_ingre.difference(combine)

        shortage_score = (len(no_match_prior) * 10) + len(no_match_main) - (len(match_extra)*0.5)
        redundancy_score = len(redundancy)
        score = shortage_score + redundancy_score

        distances.append(score)

    min_value = min(distances)
    min_indices = [i for i, value in enumerate(distances) if value == min_value]

    # predicted_food = []
    # for min_index in min_indices:
    #     predicted_food.append(df.loc[min_index, "Food"])

    return distances, min_indices[0]

def calculate_belief_merging_base(v8_distances, v9_distances, rtdetr_distances):
    max_list = []
    sum_list = []
    Gmax = []

    for index, v8_value in enumerate(v8_distances):
        v9_value = v9_distances[index]
        rtdetr_value = rtdetr_distances[index]

        max_value = min(v8_value, v9_value, rtdetr_value)
        sum_value = v8_value + v9_value + rtdetr_value
        numbers = [math.ceil(v8_value), math.ceil(v9_value), math.ceil(rtdetr_value)]
        numbers = sorted(numbers)
        gmax = ' '.join(map(str, numbers))

        max_list.append(max_value)
        sum_list.append(sum_value)
        Gmax.append(gmax)

    return max_list, sum_list, Gmax

def find_gmax_indices(arr):
    gmax_indices = [0]  # Bắt đầu với phần tử đầu tiên

    for i in range(1, len(arr)):
        for j in range(len(arr[0])):
            if arr[i][j] < arr[gmax_indices[0]][j]:
                gmax_indices = [i]
                break
            elif arr[i][j] > arr[gmax_indices[0]][j]:
                break
            elif j == len(arr[0]) - 1:  # Khi toàn bộ các phần tử đều bằng nhau
                gmax_indices.append(i)

    return gmax_indices


def belief_merging_find_foodname(max_list, sum, Gmax):
    global df

    min_value_of_maxlist = min(max_list)
    indices_of_maxlist = [index for index, value in enumerate(max_list) if value == min_value_of_maxlist]

    min_value_of_sum = min(sum)
    indices_of_sum = [index for index, value in enumerate(sum) if value == min_value_of_sum]

    gmax_values = [
        list(map(int, item.split())) for item in Gmax
    ]
    indices_of_gmax = find_gmax_indices(gmax_values)

    return indices_of_maxlist[0], indices_of_sum[0], indices_of_gmax[0]

predicted_max_belief_merging = []

def recognize_food(image_path):
    global predicted_max_belief_merging

    img = cv2.imread(image_path)
    yolov8_names, yolov8_detected_cls, yolov8_boxes = detect_ingredients(model_yolov8, img)
    yolov9_names, yolov9_detected_cls, yolov9_boxes = detect_ingredients(model_yolov9, img)
    rtdetr_names, rtdetr_detected_cls, rtdetr_boxes = detect_ingredients(model_rtdetr, img)

    v8_rm_duplicates = set(yolov8_detected_cls)
    v8_rm_duplicates = [int(number) for number in v8_rm_duplicates]
    yolov8_detected_cls = list(v8_rm_duplicates)
    v8_rm_duplicates = convertIdx2Class(v8_rm_duplicates, classes_ingre)

    v9_rm_duplicates = set(yolov9_detected_cls)
    v9_rm_duplicates = [int(number) for number in v9_rm_duplicates]
    yolov9_detected_cls = list(v9_rm_duplicates)
    v9_rm_duplicates = convertIdx2Class(v9_rm_duplicates, classes_ingre)

    rtdetr_rm_duplicates = set(rtdetr_detected_cls)
    rtdetr_rm_duplicates = [int(number) for number in rtdetr_rm_duplicates]
    rtdetr_detected_cls = list(rtdetr_rm_duplicates)
    rtdetr_rm_duplicates = convertIdx2Class(rtdetr_rm_duplicates, classes_ingre)

    v8_distances, v8_predicted_food = find_foodname_ver_narrow(v8_rm_duplicates)
    v9_distances, v9_predicted_food = find_foodname_ver_narrow(v9_rm_duplicates)
    rtdetr_distances, rtdetr_predicted_food = find_foodname_ver_narrow(rtdetr_rm_duplicates)
    print(v8_predicted_food, v9_predicted_food, rtdetr_predicted_food)

    max_list, sum_list, Gmax = calculate_belief_merging_base(v8_distances, v9_distances, rtdetr_distances)

    max_predicted_food, sum_predicted_food, gmax_predicted_food = belief_merging_find_foodname(max_list, sum_list, Gmax)
    print(max_predicted_food)

    # Kết quả
    # print("Detected Classes YOLOv8:", yolov8_detected_cls)
    # print("Detected Classes YOLOv9:", yolov9_detected_cls)
    # print("Detected Classes RTDETR:", rtdetr_detected_cls)
    # print("Predicted Food YOLOv8:", v8_predicted_food)
    # print("Predicted Food YOLOv9:", v9_predicted_food)
    # print("Predicted Food RTDETR:", rtdetr_predicted_food)
    # print("Max List:", max_list)
    # print("Sum List:", sum_list)
    # print("Gmax:", Gmax)
    # print("Max of Belief Merging Predicted Food:", max_predicted_food)
    # print("Sum of Belief Merging Predicted Food:", sum_predicted_food)
    # print("Gmax of Belief Merging Predicted Food:", gmax_predicted_food)

    predicted_max_belief_merging.append(max_predicted_food)


if __name__ == '__main__':
    images = []# tấm hình
    labels = []# nhãn
    imagePaths = []
    folders = ['bun-ca', 'hu-tieu-my-tho', 'bun-nuoc-leo', 'com-tam-long-xuyen', 'bun-hai-san-be-be',
               'banh-hoi-heo-quay', 'com-ga', 'cao-lau', 'mi-quang', 'bun-bo-hue',
               'pho-ha-noi', 'bun-muc', 'bun-moc', 'bun-dau-mam-tom']

    for k, category in enumerate(folders):
        for f in os.listdir('/content/data_test/'+category):
            imagePaths.append(['/content/data_test/' + category+'/'+f, k])

    for imagePath in imagePaths:
        data = imagePath[0]
        label = imagePath[1]

        images.append(data)
        labels.append(label)

    for img in images:
        recognize_food(img)

In [ ]:
print(predicted_max_belief_merging)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 8, 1, 1, 1, 1, 8, 1, 1, 1, 1, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 0, 0, 0, 0, 2, 11, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 0, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 0, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 8, 8, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 5, 8, 8, 8, 8, 7, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 9, 9, 10, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, classification_report

# Tính các chỉ số đánh giá
accuracy = accuracy_score(labels, predicted_max_belief_merging)
precision = precision_score(labels, predicted_max_belief_merging, average='weighted')
recall = recall_score(labels, predicted_max_belief_merging, average='weighted')
f1 = f1_score(labels, predicted_max_belief_merging, average='weighted')

# In ra kết quả
print('Accuracy:', accuracy * 100)
print('Precision:', precision * 100)
print('Recall:', recall * 100)
print('F1 Score:', f1* 100)

# In báo cáo chi tiết
print('\nClassification Report:')
print(classification_report(labels, predicted_max_belief_merging, target_names=folders))

# Tính accuracy cho từng nhãn
conf_matrix = confusion_matrix(labels, predicted_max_belief_merging)
class_accuracies = conf_matrix.diagonal() / conf_matrix.sum(axis=1)

print('\nAccuracy for each class:')
for class_name, class_accuracy in zip(folders, class_accuracies):
    print(f'{class_name}: {class_accuracy:.2f}')

Accuracy: 95.0
Precision: 95.96675105556683
Recall: 95.0
F1 Score: 95.10498079181137

Classification Report:
                    precision    recall  f1-score   support

            bun-ca       0.67      1.00      0.80        20
    hu-tieu-my-tho       1.00      0.86      0.92        14
      bun-nuoc-leo       1.00      0.73      0.84        26
com-tam-long-xuyen       1.00      1.00      1.00        36
 bun-hai-san-be-be       1.00      0.93      0.97        30
 banh-hoi-heo-quay       0.94      1.00      0.97        15
            com-ga       1.00      1.00      1.00        40
           cao-lau       0.97      0.95      0.96        37
          mi-quang       0.89      0.94      0.91        33
        bun-bo-hue       1.00      0.97      0.99        36
        pho-ha-noi       0.97      1.00      0.99        36
           bun-muc       0.92      0.92      0.92        13
           bun-moc       0.95      1.00      0.97        19
   bun-dau-mam-tom       1.00      0.92      0.96 

# Belief Merging of Sum

In [ ]:
from ultralytics import YOLO, RTDETR
import cv2
import math
import pandas as pd
from unidecode import unidecode
import numpy as np
import os

# Đọc file cấu hình
classes_path = '/content/drive/MyDrive/ViFLAVOR_Belief-merging/classes.txt'
excel_file_path = '/content/drive/MyDrive/ViFLAVOR_Belief-merging/rules.xlsx'

# Tải các mô hình
model_yolov8 = YOLO('/content/drive/MyDrive/ViFLAVOR_Belief-merging/yolov8_best_862.pt')
model_yolov9 = YOLO('/content/drive/MyDrive/ViFLAVOR_Belief-merging/yolov9_best_85.pt')
model_rtdetr = RTDETR('/content/drive/MyDrive/ViFLAVOR_Belief-merging/rt-detr_best_865.pt')

df = pd.read_excel(excel_file_path)
classes_ingre = []

with open(classes_path, 'r', encoding='utf-8') as file:
    lines = [line.strip() for line in file]
    classes_ingre = lines

def convertIdx2Class(set_index, classes):
    result_set = {classes[item] for item in set_index}
    return result_set

def preprocess_dataframe():
    global df
    normalized_df = df.drop(df.columns[[0, -1]], axis=1)

    for index, row in normalized_df.iterrows():
        for col in normalized_df.columns:
            value = row[col]
            if pd.notna(value):
                name_without_accents = unidecode(value)
                values = name_without_accents.split(', ')
                for i in range(len(values)):
                    values[i] = values[i].replace(' ', '-')
                row[col] = ', '.join(values)

    return normalized_df

normalized_df = preprocess_dataframe()

def detect_ingredients(model, img):
    results = model(img, conf=0.3, save=False)
    names = results[0].names
    detected_cls = results[0].boxes.cls.tolist()
    boxes = results[0].boxes.xyxy.tolist()
    return names, detected_cls, boxes

def find_foodname_ver_narrow(set_detected_ingre):
    global normalized_df
    global df
    distances = []

    for index, row in normalized_df.iterrows():
        values = {}
        values["prior"] = set(row["Priorities"].split(', '))
        values["main"] = set(row["Ingredients"].split(', '))
        values["extra"] = set(row["Secondary Ingredients"].split(', ')) if pd.notna(row["Secondary Ingredients"]) else set()

        # Không khớp với ƯU TIÊN
        no_match_prior = values["prior"].difference(set_detected_ingre)

        # Không khớp với CHÍNH
        no_match_main = values["main"].difference(set_detected_ingre)

        # Khớp với PHỤ
        match_extra = values["extra"].intersection(set_detected_ingre)

        # Phần tử thuộc detect nhưng không thuộc bộ luật CHÍNH & PHỤ
        combine = values["prior"].union(values["main"])
        combine = combine.union(values["extra"])
        redundancy = set_detected_ingre.difference(combine)

        shortage_score = (len(no_match_prior) * 10) + len(no_match_main) - (len(match_extra)*0.5)
        redundancy_score = len(redundancy)
        score = shortage_score + redundancy_score

        distances.append(score)

    min_value = min(distances)
    min_indices = [i for i, value in enumerate(distances) if value == min_value]

    return distances, min_indices[0]

def calculate_belief_merging_base(v8_distances, v9_distances, rtdetr_distances):
    max_list = []
    sum_list = []
    Gmax = []

    for index, v8_value in enumerate(v8_distances):
        v9_value = v9_distances[index]
        rtdetr_value = rtdetr_distances[index]

        max_value = min(v8_value, v9_value, rtdetr_value)
        sum_value = v8_value + v9_value + rtdetr_value
        numbers = [math.ceil(v8_value), math.ceil(v9_value), math.ceil(rtdetr_value)]
        numbers = sorted(numbers)
        gmax = ' '.join(map(str, numbers))

        max_list.append(max_value)
        sum_list.append(sum_value)
        Gmax.append(gmax)

    return max_list, sum_list, Gmax

def find_gmax_indices(arr):
    gmax_indices = [0]  # Bắt đầu với phần tử đầu tiên

    for i in range(1, len(arr)):
        for j in range(len(arr[0])):
            if arr[i][j] < arr[gmax_indices[0]][j]:
                gmax_indices = [i]
                break
            elif arr[i][j] > arr[gmax_indices[0]][j]:
                break
            elif j == len(arr[0]) - 1:  # Khi toàn bộ các phần tử đều bằng nhau
                gmax_indices.append(i)

    return gmax_indices


def belief_merging_find_foodname(max_list, sum, Gmax):
    global df

    min_value_of_maxlist = min(max_list)
    indices_of_maxlist = [index for index, value in enumerate(max_list) if value == min_value_of_maxlist]

    min_value_of_sum = min(sum)
    indices_of_sum = [index for index, value in enumerate(sum) if value == min_value_of_sum]

    gmax_values = [
        list(map(int, item.split())) for item in Gmax
    ]
    indices_of_gmax = find_gmax_indices(gmax_values)

    return indices_of_maxlist[0], indices_of_sum[0], indices_of_gmax[0]

predicted_sum_belief_merging = []

def recognize_food(image_path):
    global predicted_sum_belief_merging

    img = cv2.imread(image_path)
    yolov8_names, yolov8_detected_cls, yolov8_boxes = detect_ingredients(model_yolov8, img)
    yolov9_names, yolov9_detected_cls, yolov9_boxes = detect_ingredients(model_yolov9, img)
    rtdetr_names, rtdetr_detected_cls, rtdetr_boxes = detect_ingredients(model_rtdetr, img)

    v8_rm_duplicates = set(yolov8_detected_cls)
    v8_rm_duplicates = [int(number) for number in v8_rm_duplicates]
    yolov8_detected_cls = list(v8_rm_duplicates)
    v8_rm_duplicates = convertIdx2Class(v8_rm_duplicates, classes_ingre)

    v9_rm_duplicates = set(yolov9_detected_cls)
    v9_rm_duplicates = [int(number) for number in v9_rm_duplicates]
    yolov9_detected_cls = list(v9_rm_duplicates)
    v9_rm_duplicates = convertIdx2Class(v9_rm_duplicates, classes_ingre)

    rtdetr_rm_duplicates = set(rtdetr_detected_cls)
    rtdetr_rm_duplicates = [int(number) for number in rtdetr_rm_duplicates]
    rtdetr_detected_cls = list(rtdetr_rm_duplicates)
    rtdetr_rm_duplicates = convertIdx2Class(rtdetr_rm_duplicates, classes_ingre)

    v8_distances, v8_predicted_food = find_foodname_ver_narrow(v8_rm_duplicates)
    v9_distances, v9_predicted_food = find_foodname_ver_narrow(v9_rm_duplicates)
    rtdetr_distances, rtdetr_predicted_food = find_foodname_ver_narrow(rtdetr_rm_duplicates)
    print(v8_predicted_food, v9_predicted_food, rtdetr_predicted_food)

    max_list, sum_list, Gmax = calculate_belief_merging_base(v8_distances, v9_distances, rtdetr_distances)

    max_predicted_food, sum_predicted_food, gmax_predicted_food = belief_merging_find_foodname(max_list, sum_list, Gmax)

    # print("Max of Belief Merging Predicted Food:", max_predicted_food)
    # print("Sum of Belief Merging Predicted Food:", sum_predicted_food)
    # print("Gmax of Belief Merging Predicted Food:", gmax_predicted_food)

    predicted_sum_belief_merging.append(sum_predicted_food)


if __name__ == '__main__':
    images = []# tấm hình
    labels = []# nhãn
    imagePaths = []
    folders = ['bun-ca', 'hu-tieu-my-tho', 'bun-nuoc-leo', 'com-tam-long-xuyen', 'bun-hai-san-be-be',
               'banh-hoi-heo-quay', 'com-ga', 'cao-lau', 'mi-quang', 'bun-bo-hue', 'pho-ha-noi',
               'bun-muc', 'bun-moc', 'bun-dau-mam-tom']

    for k, category in enumerate(folders):
        for f in os.listdir('/content/data_test/'+category):
            imagePaths.append(['/content/data_test/' + category+'/'+f, k])

    for imagePath in imagePaths:
        data = imagePath[0]
        label = imagePath[1]

        images.append(data)
        labels.append(label)

    for img in images:
        recognize_food(img)


In [ ]:
print(predicted_sum_belief_merging)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 8, 1, 1, 1, 1, 8, 1, 1, 1, 1, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 0, 0, 0, 0, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 0, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 0, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 8, 7, 7, 7, 7, 7, 7, 7, 7, 7, 8, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 5, 8, 8, 8, 8, 7, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 9, 9, 10, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 1

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, classification_report

# Tính các chỉ số đánh giá
accuracy = accuracy_score(labels, predicted_sum_belief_merging)
precision = precision_score(labels, predicted_sum_belief_merging, average='weighted')
recall = recall_score(labels, predicted_sum_belief_merging, average='weighted')
f1 = f1_score(labels, predicted_sum_belief_merging, average='weighted')

# In ra kết quả
print('Accuracy:', accuracy * 100)
print('Precision:', precision * 100)
print('Recall:', recall * 100)
print('F1 Score:', f1* 100)

# In báo cáo chi tiết
print('\nClassification Report:')
print(classification_report(labels, predicted_sum_belief_merging, target_names=folders))

# Tính accuracy cho từng nhãn
conf_matrix = confusion_matrix(labels, predicted_sum_belief_merging)
class_accuracies = conf_matrix.diagonal() / conf_matrix.sum(axis=1)

print('\nAccuracy for each class:')
for class_name, class_accuracy in zip(folders, class_accuracies):
    print(f'{class_name}: {class_accuracy:.2f}')

Accuracy: 94.47368421052632
Precision: 95.59807917650716
Recall: 94.47368421052632
F1 Score: 94.63417173516093

Classification Report:
                    precision    recall  f1-score   support

            bun-ca       0.67      1.00      0.80        20
    hu-tieu-my-tho       1.00      0.86      0.92        14
      bun-nuoc-leo       1.00      0.77      0.87        26
com-tam-long-xuyen       1.00      1.00      1.00        36
 bun-hai-san-be-be       1.00      0.93      0.97        30
 banh-hoi-heo-quay       0.94      1.00      0.97        15
            com-ga       1.00      1.00      1.00        40
           cao-lau       0.97      0.86      0.91        37
          mi-quang       0.82      0.94      0.87        33
        bun-bo-hue       1.00      0.97      0.99        36
        pho-ha-noi       0.97      1.00      0.99        36
           bun-muc       1.00      0.92      0.96        13
           bun-moc       0.95      1.00      0.97        19
   bun-dau-mam-tom      

# Belief Merging of Gmax

In [ ]:
from ultralytics import YOLO, RTDETR
import cv2
import math
import pandas as pd
from unidecode import unidecode
import numpy as np
import os

# Đọc file cấu hình
classes_path = '/content/drive/MyDrive/ViFLAVOR_Belief-merging/classes.txt'
excel_file_path = '/content/drive/MyDrive/ViFLAVOR_Belief-merging/rules.xlsx'

# Tải các mô hình
model_yolov8 = YOLO('/content/drive/MyDrive/ViFLAVOR_Belief-merging/yolov8_best_862.pt')
model_yolov9 = YOLO('/content/drive/MyDrive/ViFLAVOR_Belief-merging/yolov9_best_85.pt')
model_rtdetr = RTDETR('/content/drive/MyDrive/ViFLAVOR_Belief-merging/rt-detr_best_865.pt')

df = pd.read_excel(excel_file_path)
classes_ingre = []

with open(classes_path, 'r', encoding='utf-8') as file:
    lines = [line.strip() for line in file]
    classes_ingre = lines

def convertIdx2Class(set_index, classes):
    result_set = {classes[item] for item in set_index}
    return result_set

def preprocess_dataframe():
    global df
    normalized_df = df.drop(df.columns[[0, -1]], axis=1)

    for index, row in normalized_df.iterrows():
        for col in normalized_df.columns:
            value = row[col]
            if pd.notna(value):
                name_without_accents = unidecode(value)
                values = name_without_accents.split(', ')
                for i in range(len(values)):
                    values[i] = values[i].replace(' ', '-')
                row[col] = ', '.join(values)

    return normalized_df

normalized_df = preprocess_dataframe()

def detect_ingredients(model, img):
    results = model(img, conf=0.3, save=False)
    names = results[0].names
    detected_cls = results[0].boxes.cls.tolist()
    boxes = results[0].boxes.xyxy.tolist()
    return names, detected_cls, boxes

def find_foodname_ver_narrow(set_detected_ingre):
    global normalized_df
    global df
    distances = []

    for index, row in normalized_df.iterrows():
        values = {}
        values["prior"] = set(row["Priorities"].split(', '))
        values["main"] = set(row["Ingredients"].split(', '))
        values["extra"] = set(row["Secondary Ingredients"].split(', ')) if pd.notna(row["Secondary Ingredients"]) else set()

        # Không khớp với ƯU TIÊN
        no_match_prior = values["prior"].difference(set_detected_ingre)

        # Không khớp với CHÍNH
        no_match_main = values["main"].difference(set_detected_ingre)

        # Khớp với PHỤ
        match_extra = values["extra"].intersection(set_detected_ingre)

        # Phần tử thuộc detect nhưng không thuộc bộ luật CHÍNH & PHỤ
        combine = values["prior"].union(values["main"])
        combine = combine.union(values["extra"])
        redundancy = set_detected_ingre.difference(combine)

        shortage_score = (len(no_match_prior) * 10) + len(no_match_main) - (len(match_extra)*0.5)
        redundancy_score = len(redundancy)
        score = shortage_score + redundancy_score

        distances.append(score)

    min_value = min(distances)
    min_indices = [i for i, value in enumerate(distances) if value == min_value]

    return distances, min_indices[0]

def calculate_belief_merging_base(v8_distances, v9_distances, rtdetr_distances):
    max_list = []
    sum_list = []
    Gmax = []

    for index, v8_value in enumerate(v8_distances):
        v9_value = v9_distances[index]
        rtdetr_value = rtdetr_distances[index]

        max_value = min(v8_value, v9_value, rtdetr_value)
        sum_value = v8_value + v9_value + rtdetr_value
        numbers = [math.ceil(v8_value), math.ceil(v9_value), math.ceil(rtdetr_value)]
        numbers = sorted(numbers)
        gmax = ' '.join(map(str, numbers))

        max_list.append(max_value)
        sum_list.append(sum_value)
        Gmax.append(gmax)

    return max_list, sum_list, Gmax

def find_gmax_indices(arr):
    gmax_indices = [0]  # Bắt đầu với phần tử đầu tiên

    for i in range(1, len(arr)):
        for j in range(len(arr[0])):
            if arr[i][j] < arr[gmax_indices[0]][j]:
                gmax_indices = [i]
                break
            elif arr[i][j] > arr[gmax_indices[0]][j]:
                break
            elif j == len(arr[0]) - 1:  # Khi toàn bộ các phần tử đều bằng nhau
                gmax_indices.append(i)

    return gmax_indices


def belief_merging_find_foodname(max_list, sum, Gmax):
    global df

    min_value_of_maxlist = min(max_list)
    indices_of_maxlist = [index for index, value in enumerate(max_list) if value == min_value_of_maxlist]

    min_value_of_sum = min(sum)
    indices_of_sum = [index for index, value in enumerate(sum) if value == min_value_of_sum]

    gmax_values = [
        list(map(int, item.split())) for item in Gmax
    ]
    indices_of_gmax = find_gmax_indices(gmax_values)

    return indices_of_maxlist[0], indices_of_sum[0], indices_of_gmax[0]

predicted_gmax_belief_merging = []

def recognize_food(image_path):
    global predicted_gmax_belief_merging

    img = cv2.imread(image_path)
    yolov8_names, yolov8_detected_cls, yolov8_boxes = detect_ingredients(model_yolov8, img)
    yolov9_names, yolov9_detected_cls, yolov9_boxes = detect_ingredients(model_yolov9, img)
    rtdetr_names, rtdetr_detected_cls, rtdetr_boxes = detect_ingredients(model_rtdetr, img)

    v8_rm_duplicates = set(yolov8_detected_cls)
    v8_rm_duplicates = [int(number) for number in v8_rm_duplicates]
    yolov8_detected_cls = list(v8_rm_duplicates)
    v8_rm_duplicates = convertIdx2Class(v8_rm_duplicates, classes_ingre)

    v9_rm_duplicates = set(yolov9_detected_cls)
    v9_rm_duplicates = [int(number) for number in v9_rm_duplicates]
    yolov9_detected_cls = list(v9_rm_duplicates)
    v9_rm_duplicates = convertIdx2Class(v9_rm_duplicates, classes_ingre)

    rtdetr_rm_duplicates = set(rtdetr_detected_cls)
    rtdetr_rm_duplicates = [int(number) for number in rtdetr_rm_duplicates]
    rtdetr_detected_cls = list(rtdetr_rm_duplicates)
    rtdetr_rm_duplicates = convertIdx2Class(rtdetr_rm_duplicates, classes_ingre)

    v8_distances, v8_predicted_food = find_foodname_ver_narrow(v8_rm_duplicates)
    v9_distances, v9_predicted_food = find_foodname_ver_narrow(v9_rm_duplicates)
    rtdetr_distances, rtdetr_predicted_food = find_foodname_ver_narrow(rtdetr_rm_duplicates)

    max_list, sum_list, Gmax = calculate_belief_merging_base(v8_distances, v9_distances, rtdetr_distances)

    max_predicted_food, sum_predicted_food, gmax_predicted_food = belief_merging_find_foodname(max_list, sum_list, Gmax)

    # print("Max of Belief Merging Predicted Food:", max_predicted_food)
    # print("Sum of Belief Merging Predicted Food:", sum_predicted_food)
    # print("Gmax of Belief Merging Predicted Food:", gmax_predicted_food)

    predicted_gmax_belief_merging.append(gmax_predicted_food)


if __name__ == '__main__':
    images = []# tấm hình
    labels = []# nhãn
    imagePaths = []
    folders = ['bun-ca', 'hu-tieu-my-tho', 'bun-nuoc-leo', 'com-tam-long-xuyen', 'bun-hai-san-be-be',
               'banh-hoi-heo-quay', 'com-ga', 'cao-lau', 'mi-quang', 'bun-bo-hue', 'pho-ha-noi',
               'bun-muc', 'bun-moc', 'bun-dau-mam-tom']

    for k, category in enumerate(folders):
        for f in os.listdir('/content/data_test/'+category):
            imagePaths.append(['/content/data_test/' + category+'/'+f, k])

    for imagePath in imagePaths:
        data = imagePath[0]
        label = imagePath[1]

        images.append(data)
        labels.append(label)

    for img in images:
        recognize_food(img)


In [ ]:
print(predicted_gmax_belief_merging)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 8, 1, 1, 1, 8, 1, 1, 1, 1, 2, 2, 0, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 0, 2, 2, 2, 2, 0, 0, 0, 0, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 0, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 0, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 8, 8, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 5, 8, 8, 8, 8, 7, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 9, 9, 10, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 1

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, classification_report

# Tính các chỉ số đánh giá
accuracy = accuracy_score(labels, predicted_gmax_belief_merging)
precision = precision_score(labels, predicted_gmax_belief_merging, average='weighted')
recall = recall_score(labels, predicted_gmax_belief_merging, average='weighted')
f1 = f1_score(labels, predicted_gmax_belief_merging, average='weighted')

# In ra kết quả
print('Accuracy:', accuracy * 100)
print('Precision:', precision * 100)
print('Recall:', recall * 100)
print('F1 Score:', f1* 100)

# In báo cáo chi tiết
print('\nClassification Report:')
print(classification_report(labels, predicted_gmax_belief_merging, target_names=folders))

# Tính accuracy cho từng nhãn
conf_matrix = confusion_matrix(labels, predicted_gmax_belief_merging)
class_accuracies = conf_matrix.diagonal() / conf_matrix.sum(axis=1)

print('\nAccuracy for each class:')
for class_name, class_accuracy in zip(folders, class_accuracies):
    print(f'{class_name}: {class_accuracy:.2f}')

Accuracy: 94.73684210526315
Precision: 95.84449045411884
Recall: 94.73684210526315
F1 Score: 94.87129663556867

Classification Report:
                    precision    recall  f1-score   support

            bun-ca       0.65      1.00      0.78        20
    hu-tieu-my-tho       1.00      0.86      0.92        14
      bun-nuoc-leo       1.00      0.73      0.84        26
com-tam-long-xuyen       1.00      1.00      1.00        36
 bun-hai-san-be-be       0.97      0.93      0.95        30
 banh-hoi-heo-quay       0.94      1.00      0.97        15
            com-ga       1.00      1.00      1.00        40
           cao-lau       0.97      0.95      0.96        37
          mi-quang       0.89      0.94      0.91        33
        bun-bo-hue       1.00      0.97      0.99        36
        pho-ha-noi       0.97      1.00      0.99        36
           bun-muc       1.00      0.85      0.92        13
           bun-moc       0.95      1.00      0.97        19
   bun-dau-mam-tom      

# Evaluating to Yolov8

In [ ]:
from ultralytics import YOLO, RTDETR
import cv2
import math
import pandas as pd
from unidecode import unidecode
import numpy as np
import os

# Đọc file cấu hình
classes_path = '/content/classes.txt'
excel_file_path = '/content/rules.xlsx'

# Tải các mô hình
model_yolov8 = YOLO('/content/drive/MyDrive/My_Research/ViFLAVOR_Belief-merging/models/detector/yolov8_best_862.pt')

df = pd.read_excel(excel_file_path)
classes_ingre = []

with open(classes_path, 'r', encoding='utf-8') as file:
    lines = [line.strip() for line in file]
    classes_ingre = lines

def convertIdx2Class(set_index, classes):
    result_set = {classes[item] for item in set_index}
    return result_set

def preprocess_dataframe():
    global df
    normalized_df = df.drop(df.columns[[0, -1]], axis=1)

    for index, row in normalized_df.iterrows():
        for col in normalized_df.columns:
            value = row[col]
            if pd.notna(value):
                name_without_accents = unidecode(value)
                values = name_without_accents.split(', ')
                for i in range(len(values)):
                    values[i] = values[i].replace(' ', '-')
                row[col] = ', '.join(values)

    return normalized_df

normalized_df = preprocess_dataframe()

def detect_ingredients(model, img):
    results = model(img, conf=0.3, save=False)
    names = results[0].names
    detected_cls = results[0].boxes.cls.tolist()
    boxes = results[0].boxes.xyxy.tolist()
    return names, detected_cls, boxes

v8_2D2 = []

def find_foodname_ver_narrow(set_detected_ingre):
    global normalized_df
    global df
    distances = []

    for index, row in normalized_df.iterrows():
        values = {}
        values["prior"] = set(row["Priorities"].split(', '))
        values["main"] = set(row["Ingredients"].split(', '))
        values["extra"] = set(row["Secondary Ingredients"].split(', ')) if pd.notna(row["Secondary Ingredients"]) else set()

        # Không khớp với ƯU TIÊN
        no_match_prior = values["prior"].difference(set_detected_ingre)

        # Không khớp với CHÍNH
        no_match_main = values["main"].difference(set_detected_ingre)

        # Khớp với PHỤ
        match_extra = values["extra"].intersection(set_detected_ingre)

        # Phần tử thuộc detect nhưng không thuộc bộ luật CHÍNH & PHỤ
        combine = values["prior"].union(values["main"])
        combine = combine.union(values["extra"])
        redundancy = set_detected_ingre.difference(combine)

        shortage_score = (len(no_match_prior) * 10) + len(no_match_main) - (len(match_extra)*0.5)
        redundancy_score = len(redundancy)
        score = shortage_score + redundancy_score

        distances.append(score)

    min_value = min(distances)
    min_indices = [i for i, value in enumerate(distances) if value == min_value]
    return distances, min_indices[0]


def recognize_food(image_path):
    global v8_2D2
    img = cv2.imread(image_path)
    yolov8_names, yolov8_detected_cls, yolov8_boxes = detect_ingredients(model_yolov8, img)
    v8_rm_duplicates = set(yolov8_detected_cls)
    v8_rm_duplicates = [int(number) for number in v8_rm_duplicates]
    yolov8_detected_cls = list(v8_rm_duplicates)
    v8_rm_duplicates = convertIdx2Class(v8_rm_duplicates, classes_ingre)

    v8_distances, v8_predicted_food = find_foodname_ver_narrow(v8_rm_duplicates)
    print(v8_predicted_food)
    v8_2D2.append(v8_predicted_food)


if __name__ == '__main__':
    images = []# tấm hình
    labels = []# nhãn
    imagePaths = []
    folders = ['bun-ca', 'hu-tieu-my-tho', 'bun-nuoc-leo', 'com-tam-long-xuyen', 'bun-hai-san-be-be',
               'banh-hoi-heo-quay', 'com-ga', 'cao-lau', 'mi-quang', 'bun-bo-hue', 'pho-ha-noi',
               'bun-muc', 'bun-moc', 'bun-dau-mam-tom']

    for k, category in enumerate(folders):
        for f in os.listdir('/content/drive/MyDrive/My_Research/ViFLAVOR_Belief-merging/dataset_classification/data_test/'+category):
            imagePaths.append(['/content/drive/MyDrive/My_Research/ViFLAVOR_Belief-merging/dataset_classification/data_test/' + category+'/'+f, k])

    for imagePath in imagePaths:
        data = imagePath[0]
        label = imagePath[1]

        images.append(data)
        labels.append(label)

    for img in images:
        recognize_food(img)


In [ ]:
print(v8_2D2)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 8, 8, 11, 1, 1, 1, 1, 1, 1, 1, 0, 2, 2, 2, 0, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 0, 0, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 0, 4, 4, 4, 4, 4, 4, 0, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 8, 8, 7, 7, 7, 7, 7, 7, 8, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 7, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 5, 8, 8, 8, 8, 9, 9, 9, 10, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 9, 10, 10, 10, 10, 10, 10, 10, 1

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, classification_report

# Tính các chỉ số đánh giá
accuracy = accuracy_score(labels, v8_2D2)
precision = precision_score(labels, v8_2D2, average='weighted')
recall = recall_score(labels, v8_2D2, average='weighted')
f1 = f1_score(labels, v8_2D2, average='weighted')

# In ra kết quả
print('Accuracy:', accuracy * 100)
print('Precision:', precision * 100)
print('Recall:', recall * 100)
print('F1 Score:', f1* 100)

# In báo cáo chi tiết
print('\nClassification Report:')
print(classification_report(labels, v8_2D2, target_names=folders))

# Tính accuracy cho từng nhãn
conf_matrix = confusion_matrix(labels, v8_2D2)
class_accuracies = conf_matrix.diagonal() / conf_matrix.sum(axis=1)

print('\nAccuracy for each class:')
for class_name, class_accuracy in zip(folders, class_accuracies):
    print(f'{class_name}: {class_accuracy:.2f}')

Accuracy: 94.21052631578948
Precision: 95.36190779367773
Recall: 94.21052631578948
F1 Score: 94.34501455419041

Classification Report:
                    precision    recall  f1-score   support

            bun-ca       0.65      1.00      0.78        20
    hu-tieu-my-tho       1.00      0.71      0.83        14
      bun-nuoc-leo       1.00      0.77      0.87        26
com-tam-long-xuyen       1.00      1.00      1.00        36
 bun-hai-san-be-be       1.00      0.93      0.97        30
 banh-hoi-heo-quay       0.94      1.00      0.97        15
            com-ga       1.00      1.00      1.00        40
           cao-lau       0.97      0.92      0.94        37
          mi-quang       0.86      0.94      0.90        33
        bun-bo-hue       0.97      0.97      0.97        36
        pho-ha-noi       0.97      0.97      0.97        36
           bun-muc       0.92      0.92      0.92        13
           bun-moc       0.95      1.00      0.97        19
   bun-dau-mam-tom      

# Evaluating to Yolov9

In [ ]:
from ultralytics import YOLO, RTDETR
import cv2
import math
import pandas as pd
from unidecode import unidecode
import numpy as np
import os

# Đọc file cấu hình
classes_path = '/content/classes.txt'
excel_file_path = '/content/rules.xlsx'

# Tải các mô hình
model_yolov9 = YOLO('/content/drive/MyDrive/My_Research/ViFLAVOR_Belief-merging/models/detector/yolov9_best_85.pt')

df = pd.read_excel(excel_file_path)
classes_ingre = []

with open(classes_path, 'r', encoding='utf-8') as file:
    lines = [line.strip() for line in file]
    classes_ingre = lines

def convertIdx2Class(set_index, classes):
    result_set = {classes[item] for item in set_index}
    return result_set

def preprocess_dataframe():
    global df
    normalized_df = df.drop(df.columns[[0, -1]], axis=1)

    for index, row in normalized_df.iterrows():
        for col in normalized_df.columns:
            value = row[col]
            if pd.notna(value):
                name_without_accents = unidecode(value)
                values = name_without_accents.split(', ')
                for i in range(len(values)):
                    values[i] = values[i].replace(' ', '-')
                row[col] = ', '.join(values)

    return normalized_df

normalized_df = preprocess_dataframe()

def detect_ingredients(model, img):
    results = model(img, conf=0.3, save=False)
    names = results[0].names
    detected_cls = results[0].boxes.cls.tolist()
    boxes = results[0].boxes.xyxy.tolist()
    return names, detected_cls, boxes

v9_2D2 = []

def find_foodname_ver_narrow(set_detected_ingre):
    global normalized_df
    global df
    distances = []

    for index, row in normalized_df.iterrows():
        values = {}
        values["prior"] = set(row["Priorities"].split(', '))
        values["main"] = set(row["Ingredients"].split(', '))
        values["extra"] = set(row["Secondary Ingredients"].split(', ')) if pd.notna(row["Secondary Ingredients"]) else set()

        # Không khớp với ƯU TIÊN
        no_match_prior = values["prior"].difference(set_detected_ingre)

        # Không khớp với CHÍNH
        no_match_main = values["main"].difference(set_detected_ingre)

        # Khớp với PHỤ
        match_extra = values["extra"].intersection(set_detected_ingre)

        # Phần tử thuộc detect nhưng không thuộc bộ luật CHÍNH & PHỤ
        combine = values["prior"].union(values["main"])
        combine = combine.union(values["extra"])
        redundancy = set_detected_ingre.difference(combine)

        shortage_score = (len(no_match_prior) * 10) + len(no_match_main) - (len(match_extra)*0.5)
        redundancy_score = len(redundancy)
        score = shortage_score + redundancy_score

        distances.append(score)

    min_value = min(distances)
    min_indices = [i for i, value in enumerate(distances) if value == min_value]
    return distances, min_indices[0]


def recognize_food(image_path):
    global v9_2D2
    img = cv2.imread(image_path)
    yolov9_names, yolov9_detected_cls, yolov9_boxes = detect_ingredients(model_yolov9, img)
    v9_rm_duplicates = set(yolov9_detected_cls)
    v9_rm_duplicates = [int(number) for number in v9_rm_duplicates]
    yolov9_detected_cls = list(v9_rm_duplicates)
    v9_rm_duplicates = convertIdx2Class(v9_rm_duplicates, classes_ingre)

    v9_distances, v9_predicted_food = find_foodname_ver_narrow(v9_rm_duplicates)
    print(v9_predicted_food)
    v9_2D2.append(v9_predicted_food)


if __name__ == '__main__':
    images = []# tấm hình
    labels = []# nhãn
    imagePaths = []
    folders = ['bun-ca', 'hu-tieu-my-tho', 'bun-nuoc-leo', 'com-tam-long-xuyen', 'bun-hai-san-be-be',
               'banh-hoi-heo-quay', 'com-ga', 'cao-lau', 'mi-quang', 'bun-bo-hue', 'pho-ha-noi',
               'bun-muc', 'bun-moc', 'bun-dau-mam-tom']

    for k, category in enumerate(folders):
        for f in os.listdir('/content/drive/MyDrive/My_Research/ViFLAVOR_Belief-merging/dataset_classification/data_test/'+category):
            imagePaths.append(['/content/drive/MyDrive/My_Research/ViFLAVOR_Belief-merging/dataset_classification/data_test/' + category+'/'+f, k])

    for imagePath in imagePaths:
        data = imagePath[0]
        label = imagePath[1]

        images.append(data)
        labels.append(label)

    for img in images:
        recognize_food(img)
    print(v9_2D2)



In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, classification_report

# Tính các chỉ số đánh giá
accuracy = accuracy_score(labels, v9_2D2)
precision = precision_score(labels, v9_2D2, average='weighted')
recall = recall_score(labels, v9_2D2, average='weighted')
f1 = f1_score(labels, v9_2D2, average='weighted')

# In ra kết quả
print('Accuracy:', accuracy * 100)
print('Precision:', precision * 100)
print('Recall:', recall * 100)
print('F1 Score:', f1* 100)

# In báo cáo chi tiết
print('\nClassification Report:')
print(classification_report(labels, v9_2D2, target_names=folders))

# Tính accuracy cho từng nhãn
conf_matrix = confusion_matrix(labels, v9_2D2)
class_accuracies = conf_matrix.diagonal() / conf_matrix.sum(axis=1)

print('\nAccuracy for each class:')
for class_name, class_accuracy in zip(folders, class_accuracies):
    print(f'{class_name}: {class_accuracy:.2f}')

Accuracy: 94.73684210526315
Precision: 95.68080449535817
Recall: 94.73684210526315
F1 Score: 94.80000812820455

Classification Report:
                    precision    recall  f1-score   support

            bun-ca       0.69      1.00      0.82        20
    hu-tieu-my-tho       1.00      0.79      0.88        14
      bun-nuoc-leo       1.00      0.73      0.84        26
com-tam-long-xuyen       1.00      1.00      1.00        36
 bun-hai-san-be-be       1.00      0.93      0.97        30
 banh-hoi-heo-quay       0.94      1.00      0.97        15
            com-ga       1.00      1.00      1.00        40
           cao-lau       0.97      0.92      0.94        37
          mi-quang       0.86      0.94      0.90        33
        bun-bo-hue       0.95      1.00      0.97        36
        pho-ha-noi       1.00      0.94      0.97        36
           bun-muc       0.87      1.00      0.93        13
           bun-moc       1.00      1.00      1.00        19
   bun-dau-mam-tom      

# Evaluating to RT-DETR

In [ ]:
from ultralytics import RTDETR
import cv2
import math
import pandas as pd
from unidecode import unidecode
import numpy as np
import os

# Đọc file cấu hình
classes_path = '/content/classes.txt'
excel_file_path = '/content/rules.xlsx'

# Tải các mô hình
model_rtdetr = RTDETR('/content/drive/MyDrive/My_Research/ViFLAVOR_Belief-merging/models/detector/rt-detr_best_865.pt')

df = pd.read_excel(excel_file_path)
classes_ingre = []

with open(classes_path, 'r', encoding='utf-8') as file:
    lines = [line.strip() for line in file]
    classes_ingre = lines

def convertIdx2Class(set_index, classes):
    result_set = {classes[item] for item in set_index}
    return result_set

def preprocess_dataframe():
    global df
    normalized_df = df.drop(df.columns[[0, -1]], axis=1)

    for index, row in normalized_df.iterrows():
        for col in normalized_df.columns:
            value = row[col]
            if pd.notna(value):
                name_without_accents = unidecode(value)
                values = name_without_accents.split(', ')
                for i in range(len(values)):
                    values[i] = values[i].replace(' ', '-')
                row[col] = ', '.join(values)

    return normalized_df

normalized_df = preprocess_dataframe()

def detect_ingredients(model, img):
    results = model(img, conf=0.3, save=False)
    names = results[0].names
    detected_cls = results[0].boxes.cls.tolist()
    boxes = results[0].boxes.xyxy.tolist()
    return names, detected_cls, boxes

rtdetr_2D2 = []

def find_foodname_ver_narrow(set_detected_ingre):
    global normalized_df
    global df
    distances = []

    for index, row in normalized_df.iterrows():
        values = {}
        values["prior"] = set(row["Priorities"].split(', '))
        values["main"] = set(row["Ingredients"].split(', '))
        values["extra"] = set(row["Secondary Ingredients"].split(', ')) if pd.notna(row["Secondary Ingredients"]) else set()

        # Không khớp với ƯU TIÊN
        no_match_prior = values["prior"].difference(set_detected_ingre)

        # Không khớp với CHÍNH
        no_match_main = values["main"].difference(set_detected_ingre)

        # Khớp với PHỤ
        match_extra = values["extra"].intersection(set_detected_ingre)

        # Phần tử thuộc detect nhưng không thuộc bộ luật CHÍNH & PHỤ
        combine = values["prior"].union(values["main"])
        combine = combine.union(values["extra"])
        redundancy = set_detected_ingre.difference(combine)

        shortage_score = (len(no_match_prior) * 10) + len(no_match_main) - (len(match_extra)*0.5)
        redundancy_score = len(redundancy)
        score = shortage_score + redundancy_score

        distances.append(score)

    min_value = min(distances)
    min_indices = [i for i, value in enumerate(distances) if value == min_value]
    return distances, min_indices[0]


def recognize_food(image_path):
    global rtdetr_2D2
    img = cv2.imread(image_path)
    rtdetr_names, rtdetr_detected_cls, rtdetr_boxes = detect_ingredients(model_rtdetr, img)
    rtdetr_rm_duplicates = set(rtdetr_detected_cls)
    rtdetr_rm_duplicates = [int(number) for number in rtdetr_rm_duplicates]
    rtdetr_detected_cls = list(rtdetr_rm_duplicates)
    rtdetr_rm_duplicates = convertIdx2Class(rtdetr_rm_duplicates, classes_ingre)

    rtdetr_distances, rtdetr_predicted_food = find_foodname_ver_narrow(rtdetr_rm_duplicates)
    print(rtdetr_predicted_food)
    rtdetr_2D2.append(rtdetr_predicted_food)


if __name__ == '__main__':
    images = []# tấm hình
    labels = []# nhãn
    imagePaths = []
    folders = ['bun-ca', 'hu-tieu-my-tho', 'bun-nuoc-leo', 'com-tam-long-xuyen', 'bun-hai-san-be-be',
               'banh-hoi-heo-quay', 'com-ga', 'cao-lau', 'mi-quang', 'bun-bo-hue', 'pho-ha-noi',
               'bun-muc', 'bun-moc', 'bun-dau-mam-tom']

    for k, category in enumerate(folders):
        for f in os.listdir('/content/drive/MyDrive/My_Research/ViFLAVOR_Belief-merging/dataset_classification/data_test/'+category):
            imagePaths.append(['/content/drive/MyDrive/My_Research/ViFLAVOR_Belief-merging/dataset_classification/data_test/' + category+'/'+f, k])

    for imagePath in imagePaths:
        data = imagePath[0]
        label = imagePath[1]

        images.append(data)
        labels.append(label)

    for img in images:
        recognize_food(img)


In [ ]:
print(rtdetr_2D2)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 8, 1, 1, 8, 1, 1, 1, 1, 1, 0, 2, 2, 2, 0, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 8, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 0, 4, 4, 4, 4, 4, 4, 0, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 8, 8, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 8, 7, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 7, 0, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 9, 9, 9, 7, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 0, 9, 9, 9, 9, 9, 9, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, classification_report

# Tính các chỉ số đánh giá
accuracy = accuracy_score(labels, rtdetr_2D2)
precision = precision_score(labels, rtdetr_2D2, average='weighted')
recall = recall_score(labels, rtdetr_2D2, average='weighted')
f1 = f1_score(labels, rtdetr_2D2, average='weighted')

# In ra kết quả
print('Accuracy:', accuracy * 100)
print('Precision:', precision * 100)
print('Recall:', recall * 100)
print('F1 Score:', f1* 100)

# In báo cáo chi tiết
print('\nClassification Report:')
print(classification_report(labels, rtdetr_2D2, target_names=folders))

# Tính accuracy cho từng nhãn
conf_matrix = confusion_matrix(labels, rtdetr_2D2)
class_accuracies = conf_matrix.diagonal() / conf_matrix.sum(axis=1)

print('\nAccuracy for each class:')
for class_name, class_accuracy in zip(folders, class_accuracies):
    print(f'{class_name}: {class_accuracy:.2f}')

Accuracy: 94.73684210526315
Precision: 95.93324181566966
Recall: 94.73684210526315
F1 Score: 94.97862532756365

Classification Report:
                    precision    recall  f1-score   support

            bun-ca       0.65      1.00      0.78        20
    hu-tieu-my-tho       1.00      0.86      0.92        14
      bun-nuoc-leo       1.00      0.81      0.89        26
com-tam-long-xuyen       1.00      0.97      0.99        36
 bun-hai-san-be-be       1.00      0.93      0.97        30
 banh-hoi-heo-quay       1.00      1.00      1.00        15
            com-ga       1.00      1.00      1.00        40
           cao-lau       0.94      0.92      0.93        37
          mi-quang       0.84      0.94      0.89        33
        bun-bo-hue       1.00      0.94      0.97        36
        pho-ha-noi       1.00      1.00      1.00        36
           bun-muc       1.00      0.92      0.96        13
           bun-moc       0.95      1.00      0.97        19
   bun-dau-mam-tom      